In [11]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import itertools

In [21]:
#stockin=input('Enter Stock to be evaluated')
#n=int(input("WIndow duration"))
stockin='AAPL'
n=30

ticker=yf.Ticker(stockin)
today = datetime.now()
prev_day = today - timedelta(days=1)
prev_day_str = prev_day.strftime('%Y-%m-%d')

hist = ticker.history(start=prev_day_str, end=today.strftime('%Y-%m-%d'), interval='1d')

if not hist.empty:
    prev_open = hist['Open'].iloc[0]
    prev_close = hist['Close'].iloc[0]
    print(f"Previous day open: {prev_open}")
    print(f"Previous day close: {prev_close}")
else:
    print("No data for previous day.")

Previous day open: 199.72999572753906
Previous day close: 196.4499969482422


In [22]:
prev_day_mean=(prev_close+prev_open)/2
prev_day_mean

198.08999633789062

In [23]:
df = yf.download(tickers=stockin, period='1d', interval='1m')

df.dropna(inplace=True)

df['RollingMean_n'] = df['Close'].shift(1).rolling(window=n).mean()
df['RollingStd_n'] = df['Close'].shift(1).rolling(window=n).std()

current_price = df['Close'].iloc[-1]
rolling_mean_nmin = df['RollingMean_n'].iloc[-1]
rolling_std_nmin = df['RollingStd_n'].iloc[-1]

print("Current Price:          ", current_price)
print(f"{n}-min Rolling Mean:    ", rolling_mean_nmin)
print(f"{n}-min Rolling Std Dev: ", rolling_std_nmin)

C:\Users\sauri\AppData\Local\Temp\ipykernel_1524\312912579.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=stockin, period='1d', interval='1m')
[*********************100%***********************]  1 of 1 completed

Current Price:           Ticker
AAPL    196.449997
Name: 2025-06-13 19:59:00+00:00, dtype: float64
30-min Rolling Mean:     196.13538360595703
30-min Rolling Std Dev:  0.20547488933702285


In [24]:
z_score=(current_price-rolling_mean_nmin) / rolling_std_nmin
max_vol_nmin = df['Volume'].shift(1).tail(n).max()
current_volume = df['Volume'].iloc[-1]

print("Current Tick Volume:", current_volume)
print(f"Max Volume in Last {n} Min (excluding current tick):", max_vol_nmin)
print("z-score",z_score)

Current Tick Volume: Ticker
AAPL    1003296
Name: 2025-06-13 19:59:00+00:00, dtype: int64
Max Volume in Last 30 Min (excluding current tick): Ticker
AAPL    415271.0
dtype: float64
z-score Ticker
AAPL    1.531152
Name: 2025-06-13 19:59:00+00:00, dtype: float64


In [25]:
amount = 10000
initial_amount = amount
position = 0
shares_held = 0
buy_percentage = 0.90
sell_percentage = 0.30
target_profit = initial_amount * 0.20

In [ ]:
def buy_sig(z_score, max_vol_nmin, current_volume):
    z_score = float(z_score.iloc[-1]) if hasattr(z_score, 'iloc') else float(z_score)
    max_vol_nmin = float(max_vol_nmin.iloc[-1]) if hasattr(max_vol_nmin, 'iloc') else float(max_vol_nmin)
    current_volume = float(current_volume.iloc[-1]) if hasattr(current_volume, 'iloc') else float(current_volume)
    
    signal = 0
    signal_type = "HOLD"
    
    volume_margin = abs(current_volume - max_vol_nmin) / max_vol_nmin <= 0.15
    
    if z_score <= -0.3:
        signal = 1
        signal_type = "BUY"
    elif z_score > 1.3:
        signal = -1  
        signal_type = "SELL"
    elif volume_margin and (z_score <= 1.3) and (z_score>-0.3):
            signal = -1
            signal_type = "SELL"
    
    print(f"\nSignal Analysis:")
    print(f"Z-Score: {z_score:.4f}")
    print(f"Signal: {signal} ({signal_type})")
    print(f"Current Volume: {current_volume}")
    print(f"Max Volume ({n}min): {max_vol_nmin}")
    print(f"Volume within 15% margin: {volume_margin}")
    
    return signal

In [27]:
signal = buy_sig(z_score, max_vol_nmin, current_volume)

current_price = float(current_price.iloc[-1]) if hasattr(current_price, 'iloc') else float(current_price)

if signal == 1 and position == 0:  
    investment = amount * buy_percentage
    shares_to_buy = int(investment // current_price)  
    if shares_to_buy > 0:
        position = current_price * shares_to_buy
        shares_held = shares_to_buy
        amount -= position
        print(f"Buying {shares_to_buy} shares at {current_price}")
        print(f"Position: {position}, Remaining cash: {amount}")

elif signal == -1 and position > 0:  
    if z_score > 1.8 and abs(current_volume - max_vol_nmin) / max_vol_nmin <= 0.15:
        shares_to_sell = shares_held  
    else:
        shares_to_sell = int(shares_held * sell_percentage)  
        
    if shares_to_sell > 0:
        sale_value = current_price * shares_to_sell
        amount += sale_value
        shares_held -= shares_to_sell
        position = current_price * shares_held
        print(f"Selling {shares_to_sell} shares at {current_price}")
        print(f"Position: {position}, Available cash: {amount}")

total_value = amount + position
profit = total_value - initial_amount

if amount <= 0:
    print("Trading stopped: No funds available")
elif profit >= target_profit:
    print(f"Trading stopped: Profit target reached. Total profit: {profit}")

print(f"Current total value: {total_value}")
print(f"Current profit/loss: {profit}")


Signal Analysis:
Z-Score: 1.5312
Signal: -1 (SELL)
Current Volume: 1003296.0
Max Volume (30min): 415271.0
Volume within 15% margin: False
Current total value: 10000
Current profit/loss: 0


### BACKTESTING


In [28]:
def backtest_strategy(symbol, start_date, end_date, initial_capital=10000):
    df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
    
    if df.empty:
        print('No data available for the specified period')
        return
    
    amount = initial_capital
    initial_amount = amount
    position = 0
    shares_held = 0
    trades = []
    
    df['RollingMean_n'] = df['Close'].shift(1).rolling(window=n).mean()
    df['RollingStd_n'] = df['Close'].shift(1).rolling(window=n).std()
    
    df = df.dropna()
    
    for i in range(len(df)):
        current_price = float(df['Close'].iloc[i])
        rolling_mean_nmin = df['RollingMean_n'].iloc[i]
        rolling_std_nmin = df['RollingStd_n'].iloc[i]
        
        z_score = (current_price - rolling_mean_nmin) / rolling_std_nmin
        
        if i >= n:
            recent_volumes = df['Volume'].iloc[i-n:i]
            max_vol_nmin = recent_volumes.max()
        else:
            max_vol_nmin = df['Volume'].iloc[:i].max()
        
        current_volume = df['Volume'].iloc[i]
        
        signal = buy_sig(z_score, max_vol_nmin, current_volume)
        
        if signal == 1 and position == 0:  
            investment = amount * buy_percentage
            shares_to_buy = int(investment // current_price)
            if shares_to_buy > 0:
                position = current_price * shares_to_buy
                shares_held = shares_to_buy
                amount -= position
                trades.append({
                    'time': df.index[i],
                    'type': 'BUY',
                    'shares': shares_to_buy,
                    'price': current_price,
                    'value': position,
                    'remaining_cash': amount
                })

        elif signal == -1 and position > 0:  
            shares_to_sell = max(1, int(np.ceil(shares_held * sell_percentage))) if shares_held > 0 else 0
            if shares_to_sell > 0:
                sale_value = current_price * shares_to_sell
                amount += sale_value
                shares_held -= shares_to_sell
                position = current_price * shares_held
                trades.append({
                    'time': df.index[i],
                    'type': 'SELL',
                    'shares': shares_to_sell,
                    'price': current_price,
                    'value': sale_value,
                    'remaining_cash': amount
                })
        
        if shares_held > 0:
            position = current_price * shares_held
    
    total_value = amount + position
    total_return = ((total_value - initial_capital) / initial_capital) * 100
    
    print(f'\nBacktest Results ({start_date} to {end_date}):')
    print(f'Initial Capital: ${initial_amount:,.2f}')
    print(f'Final Capital: ${total_value:,.2f}')
    print(f'Total Return: {total_return:.2f}%')
    print(f'Number of Trades: {len(trades)}')
    
    print('\nTrade History:')
    for trade in trades:
        print(f"{trade['time']} - {trade['type']}: {trade['shares']} shares @ ${trade['price']:.2f}")
    
    return df, trades

In [29]:
from datetime import datetime, timedelta

end_date = datetime.now()
start_date = end_date - timedelta(days=4)  

df, trades = backtest_strategy(stockin, start_date, end_date, initial_capital=10000)

C:\Users\sauri\AppData\Local\Temp\ipykernel_1524\2108613513.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval='1m')
[*********************100%***********************]  1 of 1 completed
C:\Users\sauri\AppData\Local\Temp\ipykernel_1524\2108613513.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df['Close'].iloc[i])



Signal Analysis:
Z-Score: -1.2161
Signal: 1 (BUY)
Current Volume: 137269.0
Max Volume (30min): nan
Volume within 15% margin: False

Signal Analysis:
Z-Score: -2.2369
Signal: 1 (BUY)
Current Volume: 196912.0
Max Volume (30min): 137269.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -2.4150
Signal: 1 (BUY)
Current Volume: 175758.0
Max Volume (30min): 196912.0
Volume within 15% margin: True

Signal Analysis:
Z-Score: -1.5964
Signal: 1 (BUY)
Current Volume: 113555.0
Max Volume (30min): 196912.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: 3.5848
Signal: -1 (SELL)
Current Volume: 129298.0
Max Volume (30min): 196912.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: -2.1183
Signal: 1 (BUY)
Current Volume: 195924.0
Max Volume (30min): 196912.0
Volume within 15% margin: True

Signal Analysis:
Z-Score: 0.3973
Signal: 0 (HOLD)
Current Volume: 239967.0
Max Volume (30min): 196912.0
Volume within 15% margin: False

Signal Analysis:
Z-Score: 1.1445
Signal: 0 (HO

### MAIN CODE RUN

In [ ]:
def is_market_open():
    now = datetime.now()
    market_start = now.replace(hour=9, minute=15, second=0)
    market_end = now.replace(hour=15, minute=30, second=0)
    return market_start <= now <= market_end and now.weekday() < 5

def wait_for_next_minute():
    """Wait until the start of the next minute"""
    now = datetime.now()
    wait_seconds = 60 - now.second
    if wait_seconds > 0:
        print(f"Synchronizing with market ticks... waiting {wait_seconds} seconds")
        time.sleep(wait_seconds)

def run_trading_strategy():
    global amount, position, shares_held
    trades = []
    
    wait_for_next_minute()
    
    while True:
        try:
            if not is_market_open():
                print("Market is closed. Waiting...")
                time.sleep(60)
                continue
            
            end_time = datetime.now().replace(second=0, microsecond=0)
            start_time = end_time - timedelta(hours=1)
            
            start_timestamp = int(start_time.timestamp())
            end_timestamp = int(end_time.timestamp())
            
            try:
                df = yf.download(
                    tickers=stockin,
                    start=start_timestamp,
                    end=end_timestamp,
                    interval='1m',
                    progress=False
                )
            except Exception as e:
                print(f"Download error: {e}")
                time.sleep(60)
                continue
            
            if df.empty:
                print("No data received, waiting for next tick...")
                time.sleep(60)
                continue

            df['RollingMean_n'] = df['Close'].shift(1).rolling(window=n).mean()
            df['RollingStd_n'] = df['Close'].shift(1).rolling(window=n).std()
            
            current_price = float(df['Close'].iloc[-1])
            rolling_mean_nmin = df['RollingMean_n'].iloc[-1]
            rolling_std_nmin = df['RollingStd_n'].iloc[-1]
            z_score = (current_price - rolling_mean_nmin) / rolling_std_nmin
            
            if len(df) < 2:
                print("Not enough data for previous tick's volume, waiting...")
                time.sleep(60)
                continue
            
            prev_volume = float(df['Volume'].iloc[-2])
            recent_volumes = df['Volume'].iloc[-21:-1]  
            non_zero_volumes = recent_volumes[recent_volumes > 0]
            max_vol_nmin = non_zero_volumes.max() if not non_zero_volumes.empty else 0
            
            if prev_volume == 0:
                print("Warning: Previous tick's volume is 0, using last non-zero volume")
                prev_volume = non_zero_volumes.iloc[-1] if not non_zero_volumes.empty else 0
            
            signal = buy_sig(z_score, max_vol_nmin, prev_volume)
            
            if signal == 1 and position == 0:  
                investment = amount * buy_percentage
                shares_to_buy = int(investment // current_price)
                if shares_to_buy > 0:
                    position = current_price * shares_to_buy
                    shares_held = shares_to_buy
                    amount -= position
                    trades.append({
                        'time': datetime.now(),
                        'type': 'BUY',
                        'shares': shares_to_buy,
                        'price': current_price,
                        'value': position,
                        'remaining_cash': amount
                    })
                    print(f"\nTRADE EXECUTED - BUY:")
                    print(f"Shares: {shares_to_buy}")
                    print(f"Price: {current_price:.2f}")
                    print(f"Position: {position:.2f}")
                    print(f"Remaining Cash: {amount:.2f}")

            elif signal == -1 and position > 0:  
                shares_to_sell = max(1, int(np.ceil(shares_held * sell_percentage))) if shares_held > 0 else 0
                if shares_to_sell > 0:
                    sale_value = current_price * shares_to_sell
                    amount += sale_value
                    shares_held -= shares_to_sell
                    position = current_price * shares_held
                    trades.append({
                        'time': datetime.now(),
                        'type': 'SELL',
                        'shares': shares_to_sell,
                        'price': current_price,
                        'value': sale_value,
                        'remaining_cash': amount
                    })
                    print(f"\nTRADE EXECUTED - SELL:")
                    print(f"Shares: {shares_to_sell}")
                    print(f"Price: {current_price:.2f}")
                    print(f"Position: {position:.2f}")
                    print(f"Available Cash: {amount:.2f}")

            total_value = amount + position
            profit = total_value - initial_amount
            
            print(f"\nPORTFOLIO STATUS:")
            print(f"Time: {datetime.now()}")
            print(f"Current Price: {current_price:.2f}")
            print(f"Shares Held: {shares_held}")
            print(f"Position Value: {position:.2f}")
            print(f"Cash: {amount:.2f}")
            print(f"Total Value: {total_value:.2f}")
            print(f"Profit/Loss: {profit:.2f}")
            print("-" * 50)
            
            if amount <= 0:
                print("\nTrading stopped: No funds available")
                break
            elif profit >= target_profit:
                print(f"\nTrading stopped: Profit target reached. Total profit: {profit:.2f}")
                break
            
            if trades:
                print("\nTRADE HISTORY:")
                for trade in trades[-3:]:  
                    print(f"{trade['time']} - {trade['type']}: {trade['shares']} shares @ {trade['price']:.2f}")
            
            wait_for_next_minute()
            
        except Exception as e:
            print(f"Error occurred: {e}")
            time.sleep(60)

run_trading_strategy()